In [ ]:
import sys, os
sys.path.append(os.path.join(sys.path[0], "code"))

In [ ]:
# Run this cell only if "from mpl_toolkits.basemap import Basemap" causes KeyError: 'PROJ_LIB'

conda_dir = "/home/USERNAME/anaconda3/envs/sinkhorn_cnf" # replace with conda environment path
proj_lib = os.path.join(conda_dir, 'bin/proj')
os.environ["PROJ_LIB"] = proj_lib

In [ ]:
import pytorch_models as models
import pytorch_samplers as samplers
import pytorch_losses as losses
import pytorch_training as training
import pytorch_utils as utils
import geodesics

import importlib
import time
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.basemap import Basemap
import torch
import torch.nn as nn
import torch.nn.functional as F
from statistics import mean
from torchdiffeq import odeint
import random
import numpy as np
from dtaidistance import dtw_ndim

importlib.reload(models)
importlib.reload(losses)
importlib.reload(samplers)
importlib.reload(training)
importlib.reload(utils)
importlib.reload(geodesics)

In [ ]:
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Import eBird data

data_file = 'data/snow_goose/ebird_snow_goose_data_basemap.npy'
data_array = np.load(data_file, allow_pickle=True)
data_list = list(data_array)

In [ ]:
start_month = 0
end_month = 6
data_list_spring = data_list[start_month:end_month]

In [ ]:
# Learn metric tensor from eBird data

seed = 0
n_samples = 500
space_dims = 2
scalar_hidden_dims = 32
matrix_hidden_dims = 32
gp_strength_phi = 1e-6
gp_strength_R = 1e0
fro_reg_strength = 1e9
identity_reg_strength = 0
lr = 1e-2
weight_decay = 1e-3
n_epochs_phi = 2000
n_epochs_R = 10000
R_model_type = "multilayer"

t0 = time.time()
ebird_learnedA, phi_list, rho_0_list, rho_1_list = training.twostep_train_wot_model(data_list_spring,
                                                                                    n_samples,
                                                                                    scalar_hidden_dims,
                                                                                    matrix_hidden_dims,
                                                                                    fro_reg_strength,
                                                                                    identity_reg_strength,
                                                                                    gp_strength_phi,
                                                                                    gp_strength_R,
                                                                                    lr,
                                                                                    weight_decay,
                                                                                    n_epochs_phi,
                                                                                    n_epochs_R,
                                                                                    seed,
                                                                                    R_model_type
                                                                                   )

t1 = time.time()
print('elapsed time: ' + str(t1-t0) + ' s.')

# Plot eigenvectors of metric
plt.figure(figsize=(20,20))
# Eigs plot params
x_lims = (0,12)
y_lims = (0,12)
n = 30
utils.eigs_quiver(ebird_learnedA, n, x_lims, y_lims)
for t in range(len(data_list_spring)):
    plt.scatter(data_list_spring[t][:,0], data_list_spring[t][:,1])

In [ ]:
# Save the learned metric

ebird_learnedA_path = "trained_models/ebird_learnedA_params.pt"
torch.save(ebird_learnedA.state_dict(), ebird_learnedA_path)

In [ ]:
# Load pretrained metric

ebird_learnedA_path = "trained_models/ebird_learnedA_params.pt"
space_dims = 2
matrix_hidden_dims = 32
ebird_learnedA = models.PSDMatrixMultiLayer(space_dims, matrix_hidden_dims).to(device)
ebird_learnedA.load_state_dict(torch.load(ebird_learnedA_path))

In [ ]:
# Carry out trajectory inference for each snow goose in Banks Island SNGO study

goose_id_list = [82901, 82902, 82905, 82906, 82907, 82908, 82909, 82910]
t_inits = [20000, 0, 0, 20000, 0, 0, 0, 0]
t_finals = [26000, 9100, 15057, 26500, 9000, 13037, 7200, 10000]
lambds = [5e1, 5e1, 1e2, 5e1, 1e2, 5e1, 5e1, 1e2]
lambds = [2.5e0*x for x in lambds]

for i, goose_id in enumerate(goose_id_list):
    fname = "data/snow_goose/movebank_goose_data_basemap_" + str(goose_id) + ".npy"
    true_path = np.load(fname, allow_pickle=True)
    t_init = t_inits[i]
    t_final = t_finals[i]
    
    x_0 = torch.from_numpy(true_path[t_init]).to(device)
    x_1 = torch.from_numpy(true_path[t_final]).to(device)
    n_times = 32
    t = torch.linspace(0, 1, n_times, device=device)
    lambd = lambds[i]
    x_t, losses = geodesics.compute_A_geodesic(x_0, x_1, ebird_learnedA, t, lambd)
    # Save the trajectory x_t
    fname = "results/snow_goose_experiments/final_goose_inferred_traj_2197-" + str(goose_id) + ".npy"
    np.save(fname, x_t.detach().cpu().numpy())

In [ ]:
# Generate straight-line trajectories as naive baseline

straight_line_paths = []

# Load each ground truth trajectory and generate straight-line path with 32 points
goose_id_list = [82901, 82902, 82905, 82906, 82907, 82908, 82909, 82910]
t_inits = [20000, 0, 0, 20000, 0, 0, 0, 0]
t_finals = [26000, 9100, 15057, 26500, 9000, 13037, 7201, 10000]
for i, goose_id in enumerate(goose_id_list):
    fname = "data/snow_goose/movebank_goose_data_basemap_" + str(goose_id) + ".npy"
    true_path = np.load(fname, allow_pickle=True)
    t_init = t_inits[i]
    t_final = t_finals[i] - 1
    n_times = 32
    t = torch.unsqueeze(torch.linspace(0, 1, n_times), dim=1)
    init_pt = true_path[t_init]
    final_pt = true_path[t_final]
    straight_path = t*final_pt + (1-t)*init_pt
    straight_line_paths.append(straight_path.numpy())

In [ ]:
# Plot the ground truth trajectories on LHS and inferred trajectories on RHS

fig = plt.figure(figsize=(30,20))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
fig.tight_layout()

llcrnrlat = 22
llcrnrlon = -172
urcrnrlat = 75
urcrnrlon = -90

m = Basemap(projection='merc',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat,
            llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon, suppress_ticks=True, rsphere=5, resolution='l')

# Draw map outline
m.drawcoastlines(ax=ax1, color='gray')
m.drawcoastlines(ax=ax2, color='gray')

ground_truth_paths = []
our_paths = []

# Now load each ground truth trajectory and plot on LHS
goose_id_list = [82901, 82902, 82905, 82906, 82907, 82908, 82909, 82910]
t_inits = [20000, 0, 0, 20000, 0, 0, 0, 0]
t_finals = [26000, 9100, 15057, 26500, 9000, 13037, 7201, 10000]
for i, goose_id in enumerate(goose_id_list):
    fname = "data/snow_goose/movebank_goose_data_basemap_" + str(goose_id) + ".npy"
    true_path = np.load(fname, allow_pickle=True)
    t_init = t_inits[i]
    t_final = t_finals[i]
    ground_truth_paths.append(true_path[t_init:t_final])
    x = true_path[t_init:t_final,0]
    y = true_path[t_init:t_final,1]
    ax1.scatter(x[::5], y[::5], marker="o", s=100)
    
# Now load each inferred trajectory and plot on RHS
goose_id_list = [82901, 82902, 82905, 82906, 82907, 82908, 82909, 82910]
for i, goose_id in enumerate(goose_id_list):
    fname = "results/snow_goose_experiments/final_goose_inferred_traj_2197-" + str(goose_id) + ".npy"
    inferred_path = np.load(fname, allow_pickle=True)
    our_paths.append(inferred_path)
    ax2.scatter(inferred_path[:,0], inferred_path[:,1], marker="o", s=100)

ax1.set_title("Ground truth", fontsize=50, fontweight="bold")
ax2.set_title("Ours", fontsize=50, fontweight="bold")

# Save figure
plt.savefig("results/sngo_traj_plots_horizontal.png", dpi=300, bbox_inches="tight")

In [ ]:
# Compute DTW distances between straight paths and ground truth

baseline_dtw = []

for i in range(len(ground_truth_paths)):
    true_path = ground_truth_paths[i]
    straight_path = straight_line_paths[i]
    baseline_dtw.append(dtw_ndim.distance(true_path, straight_path))

In [ ]:
# Compute DTW distances between our paths and ground truth

our_dtw = []

for i in range(len(ground_truth_paths)):
    true_path = ground_truth_paths[i]
    our_path = our_paths[i]
    our_dtw.append(dtw_ndim.distance(true_path, our_path))

In [ ]:
baseline_dtw

In [ ]:
our_dtw

In [ ]:
base_mean = sum(baseline_dtw)/len(baseline_dtw)
our_mean = sum(our_dtw)/len(our_dtw)
(base_mean - our_mean)/base_mean